# Santa's Uncertain Bags

In [119]:
%load_ext autotime
import numpy as np
import operator
import pandas as pd
import os
import time

os.chdir("C:\\path")

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 9.42 ms


## Generazione casuale dei pesi su RStudio

library(stringr) <br>
library(triangle)

gift = read.csv('gifts.csv', sep = ",", header = TRUE) <br>
gift\$Toy <- factor(str_extract(gift$GiftId, pattern = "[a-z]+"))

sample_ball = function(n) pmax(0, 1 + rnorm(n = n, mean = 1, sd = 0.3)) <br>
sample_bike = function(n) pmax(0, rnorm(n = n, mean = 20, sd = 10)) <br>
sample_blocks = function(n) if(n == 0) return(numeric(0)) else return(rtriangle(n = n, a = 5, c = 10, b = 20)) <br>
sample_book = function(n) rchisq(n = n, df = 2) <br>
sample_coal = function(n) 47 \* rbeta(n = n, shape1 = 0.5, shape2 = 0.5) <br>
sample_doll = function(n) rgamma(n = n, shape = 5, rate = 1) <br>
sample_gloves = function(n) runif(n = n, min = 0, max = 1) + (runif(n = n, min = 0, max = 1) < 0.3) * 3 <br>
sample_horse = function(n) pmax(0, rnorm(n = n, mean = 5, sd = 2)) <br>
sample_train = function(n) pmax(0, rnorm(n = n, mean = 10, sd = 5)) <br>

set.seed(1234)
gift\$Weight = sapply(gift\$Toy, function(x) do.call(paste("sample", x, sep = "_"), as.list(1)))

write.csv(x = gift, file = "Santa's gifts.csv")

## Upload del dataset con i pesi casuali

In [22]:
gift = pd.read_csv("Santa's gifts.csv", sep = ",", index_col = 0)
gift.head()

,GiftId,Toy,Weight
1,horse_0,horse,2.585869
2,horse_1,horse,5.554858
3,horse_2,horse,7.168882
4,horse_3,horse,0.308605
5,horse_4,horse,5.858249


time: 31.7 ms


In [23]:
print('Le osservazioni nel dataset sono: %s' %(len(gift)))

Le osservazioni nel dataset sono: 7166
time: 1.98 ms


## Rimozione dei pesi superiori a 50 lb e nulli

In [24]:
gift = gift[gift['Weight'] > 0]
print('Dopo aver elminato le osservazioni con peso nullo, le osservazioni rimanenti sono: %s' %(len(gift)))
gift = gift[gift['Weight'] < 50]
print('Dopo aver elminato le osservazioni con peso eccessivo, le osservazioni rimanenti sono: %s' %(len(gift)))

Dopo aver elminato le osservazioni con peso nullo, le osservazioni rimanenti sono: 7125
Dopo aver elminato le osservazioni con peso eccessivo, le osservazioni rimanenti sono: 7124
time: 12.4 ms


## Frequenze relative dei giocattoli

In [19]:
gift['Toy'].value_counts(normalize = True)

book      0.168445
ball      0.154408
doll      0.140371
blocks    0.140371
horse     0.138546
train     0.137704
bike      0.068782
gloves    0.028074
coal      0.023302
Name: Toy, dtype: float64

## Creazione di dataset distinti per ogni giocattolo

In [39]:
ball = gift[gift['Toy'] == 'ball'].reset_index(drop = True).set_index('GiftId')
bike = gift[gift['Toy'] == 'bike'].reset_index(drop = True).set_index('GiftId')
blocks = gift[gift['Toy'] == 'blocks'].reset_index(drop = True).set_index('GiftId')
book = gift[gift['Toy'] == 'book'].reset_index(drop = True).set_index('GiftId')
coal = gift[gift['Toy'] == 'coal'].reset_index(drop = True).set_index('GiftId')
doll = gift[gift['Toy'] == 'doll'].reset_index(drop = True).set_index('GiftId')
gloves = gift[gift['Toy'] == 'gloves'].reset_index(drop = True).set_index('GiftId')
horse = gift[gift['Toy'] == 'horse'].reset_index(drop = True).set_index('GiftId')
train = gift[gift['Toy'] == 'train'].reset_index(drop = True).set_index('GiftId')

time: 63 ms


# Algoritmo di ottimizzazione

In [40]:
#Liste in cui saranno memorizzati i sacchi. Sono divise per l'ID, il tipo e il peso dell'oggetto
big_bag_ID = []
big_bag_Toy = []
big_bag_weight = []

#Contatori che sono necessari per l'algoritmo
cont_bag = 0
oggetti_inseriti = 0
data_vuoti = 0
sacchi_fermati_dogana = 0

#Dizionario con le frequenze dei giocattoli inseriti, ovviamente inizializzato con frequenze nulle.
diz_freq = {'ball':0, 'bike':0, 'blocks':0, 'book':0, 'coal':0, 'doll':0, 'gloves':0, 'horse':0, 'train':0}
while cont_bag < 1000: #Algoritmo continua fino a 1000 sacche
    copy = {}
    copy.update(diz_freq) #Copia del dizionario
    peso_sacco = 0
    
    #Liste in cui saranno memorizzati i singoli elementi che comporranno il sacco
    bag_ID = []
    bag_Toy = []
    bag_Weight = []
    
    while peso_sacco < 50:
        min_toy = min(copy, key=copy.get) #In caso di più frequenze minime prende il primo in ordine alfabetico
        #Se un sacco ha già 3 elementi cerco di non inserire oggetti che complessivamente sono già stati estratti frequentemente
        if len(bag_ID) > 2 and (copy[min_toy] > (oggetti_inseriti)/(9 - data_vuoti)):
            break
            
        #Se nella tipologia di giocattolo meno frequente non ne esiste uno che possa mantenere il peso < 50lb passo alla tiplogia successiva    
        if min(globals()[min_toy]['Weight']) + peso_sacco > 50:
            success = 0
            while success == 0 and len(copy) > 1: 
                del copy[min_toy]
                min_toy = min(copy, key=copy.get)
                if min(globals()[min_toy]['Weight']) + peso_sacco > 50:
                    success == 0
                else:
                    success == 1
             
        if len(copy) < 2:
            break
                
                
                
        campione = globals()[min_toy].sample(1) #Estraggo un giocattolo da quelli con freq minore o che garatisca un peso < 50lb
        peso_temp = campione['Weight'].iloc[0] #Peso dell'elemento estratto
        
        #Esiste almeno un elemento della tipologia che non fa superare le 50lb, estraggo casualmente fino a quando ne trovo uno
        if peso_sacco + peso_temp > 50:
            while peso_sacco + peso_temp > 50:
                campione = globals()[min_toy].sample(1) 
                peso_temp = campione['Weight'].iloc[0] 
          
        
        
        #Inserisco le informazioni del giocattolo nelle tre liste
        if campione.index.values[0] not in bag_ID:
            bag_ID.append(campione.index.values[0])
            bag_Toy.append(campione['Toy'].iloc[0])
            bag_Weight.append(campione['Weight'].iloc[0])
            
            #Aumento la frequenza del dizionario in base alla tipologia di giocattolo aggiunta
            diz_freq[min_toy] = diz_freq[min_toy] + 1
            copy[min_toy] = copy[min_toy] + 1
            
            #Aggiorno il peso del sacco
            peso_sacco = peso_sacco + peso_temp
            oggetti_inseriti = oggetti_inseriti + 1
            
            globals()[min_toy].drop(campione.index.values[0], axis = 0, inplace = True)
            if len(globals()[min_toy]) == 0:
                data_vuoti = data_vuoti + 1
                del diz_freq[min_toy]
                del copy[min_toy]
     
    
    #Se non è possibile raggiungere almeno 3 giocattoli in qualsiasi modo, il sacco viene eliminato   
    if len(copy) == 0 and len(bag_ID) < 3:
        print('Attenzione: il sacco è stato bloccato dalla polizia del Polo Nord! Alcuni bambini non riceveranno il loro regalo :(')
        sacchi_fermati_dogana = sacchi_fermati_dogana + 1
    
    
    #Nel caso il sacco abbia almeno 3 giocattoli e non sono soddisfatte le condizioni precedenti, le liste iniziali sono aggiornate con il sacco creato 
    if len(bag_ID) > 2:
        big_bag_ID.append(bag_ID)
        big_bag_Toy.append(bag_Toy)
        big_bag_weight.append(bag_Weight)
        cont_bag = cont_bag + 1
    
    
    #Print di controllo
        print('Oggetti inseriti: %s' %(bag_ID))
        print('Sacchi fermati alla dogana: %s' %(sacchi_fermati_dogana))
        print('Sacchi riempiti: %s' %(len(big_bag_ID)))        
        print('Conteggio: %s' %(cont_bag))
        print('%s \n' %(diz_freq))

Oggetti inseriti: ['ball_29', 'bike_243', 'blocks_420', 'book_802', 'coal_15', 'doll_30', 'gloves_96', 'horse_541', 'train_307']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 1
Conteggio: 1
{'ball': 1, 'bike': 1, 'blocks': 1, 'book': 1, 'coal': 1, 'doll': 1, 'gloves': 1, 'horse': 1, 'train': 1} 

Oggetti inseriti: ['ball_972', 'bike_181', 'blocks_535', 'book_763', 'coal_130', 'doll_412', 'gloves_190', 'horse_801', 'train_468']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 2
Conteggio: 2
{'ball': 2, 'bike': 2, 'blocks': 2, 'book': 2, 'coal': 2, 'doll': 2, 'gloves': 2, 'horse': 2, 'train': 2} 

Oggetti inseriti: ['ball_260', 'bike_198', 'blocks_278', 'book_1112', 'coal_158', 'doll_356', 'gloves_57']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 3
Conteggio: 3
{'ball': 3, 'bike': 3, 'blocks': 3, 'book': 3, 'coal': 3, 'doll': 3, 'gloves': 3, 'horse': 2, 'train': 2} 

Oggetti inseriti: ['horse_173', 'train_386', 'ball_510', 'bike_92', 'blocks_336', 'book_606', 'coal_26']
Sacchi fermati a

Sacchi riempiti: 31
Conteggio: 31
{'ball': 30, 'bike': 30, 'blocks': 30, 'book': 30, 'coal': 30, 'doll': 30, 'gloves': 30, 'horse': 30, 'train': 30} 

Oggetti inseriti: ['ball_488', 'bike_85', 'blocks_326', 'book_774', 'coal_83', 'doll_744']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 32
Conteggio: 32
{'ball': 31, 'bike': 31, 'blocks': 31, 'book': 31, 'coal': 31, 'doll': 31, 'gloves': 30, 'horse': 30, 'train': 30} 

Oggetti inseriti: ['gloves_36', 'horse_913', 'train_786', 'ball_495', 'bike_5', 'blocks_926', 'book_112', 'coal_17', 'doll_404', 'gloves_107']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 33
Conteggio: 33
{'ball': 32, 'bike': 32, 'blocks': 32, 'book': 32, 'coal': 32, 'doll': 32, 'gloves': 32, 'horse': 31, 'train': 31} 

Oggetti inseriti: ['horse_108', 'train_75', 'ball_27', 'bike_153', 'blocks_796', 'book_139', 'coal_42']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 34
Conteggio: 34
{'ball': 33, 'bike': 33, 'blocks': 33, 'book': 33, 'coal': 33, 'doll': 32, 'gloves': 

Oggetti inseriti: ['horse_631', 'train_718', 'ball_908', 'bike_444', 'blocks_787', 'book_507']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 128
Conteggio: 128
{'ball': 101, 'bike': 101, 'blocks': 101, 'book': 101, 'coal': 100, 'doll': 100, 'gloves': 100, 'horse': 100, 'train': 100} 

Oggetti inseriti: ['coal_68', 'doll_385', 'gloves_136', 'horse_632']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 129
Conteggio: 129
{'ball': 101, 'bike': 101, 'blocks': 101, 'book': 101, 'coal': 101, 'doll': 101, 'gloves': 101, 'horse': 101, 'train': 100} 

Oggetti inseriti: ['train_729', 'ball_306', 'bike_455', 'blocks_991', 'book_165']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 130
Conteggio: 130
{'ball': 102, 'bike': 102, 'blocks': 102, 'book': 102, 'coal': 101, 'doll': 101, 'gloves': 101, 'horse': 101, 'train': 101} 

Oggetti inseriti: ['coal_69', 'doll_557', 'gloves_157', 'horse_137']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 131
Conteggio: 131
{'ball': 102, 'bike': 102, 'blocks': 102, '


Oggetti inseriti: ['bike_134', 'blocks_643', 'book_841', 'coal_10']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 233
Conteggio: 233
{'ball': 158, 'bike': 158, 'blocks': 158, 'book': 158, 'coal': 158, 'doll': 157, 'gloves': 157, 'horse': 157, 'train': 157} 

Oggetti inseriti: ['doll_474', 'gloves_13', 'horse_440', 'train_349', 'ball_95', 'bike_131']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 234
Conteggio: 234
{'ball': 159, 'bike': 159, 'blocks': 158, 'book': 158, 'coal': 158, 'doll': 158, 'gloves': 158, 'horse': 158, 'train': 158} 

Oggetti inseriti: ['blocks_13', 'book_813', 'coal_75', 'doll_212', 'gloves_80']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 235
Conteggio: 235
{'ball': 159, 'bike': 159, 'blocks': 159, 'book': 159, 'coal': 159, 'doll': 159, 'gloves': 159, 'horse': 158, 'train': 158} 

Oggetti inseriti: ['horse_295', 'train_557', 'ball_551', 'bike_236', 'blocks_650', 'book_91']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 236
Conteggio: 236
{'ball': 160, 'bike': 

Sacchi fermati alla dogana: 0
Sacchi riempiti: 348
Conteggio: 348
{'ball': 262, 'bike': 262, 'blocks': 262, 'book': 262, 'doll': 261, 'horse': 261, 'train': 261} 

Oggetti inseriti: ['doll_651', 'horse_229', 'train_876', 'ball_764', 'bike_161']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 349
Conteggio: 349
{'ball': 263, 'bike': 263, 'blocks': 262, 'book': 262, 'doll': 262, 'horse': 262, 'train': 262} 

Oggetti inseriti: ['blocks_945', 'book_1143', 'doll_172', 'horse_906', 'train_153', 'ball_235', 'bike_35']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 350
Conteggio: 350
{'ball': 264, 'bike': 264, 'blocks': 263, 'book': 263, 'doll': 263, 'horse': 263, 'train': 263} 

Oggetti inseriti: ['blocks_692', 'book_263', 'doll_978', 'horse_227', 'train_169', 'ball_755', 'bike_295']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 351
Conteggio: 351
{'ball': 265, 'bike': 265, 'blocks': 264, 'book': 264, 'doll': 264, 'horse': 264, 'train': 264} 

Oggetti inseriti: ['blocks_635', 'book_667', 'dol


Oggetti inseriti: ['bike_30', 'blocks_276', 'book_864', 'doll_477']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 527
Conteggio: 527
{'ball': 412, 'bike': 412, 'blocks': 412, 'book': 412, 'doll': 412, 'horse': 411, 'train': 411} 

Oggetti inseriti: ['horse_359', 'train_821', 'ball_989', 'bike_291']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 528
Conteggio: 528
{'ball': 413, 'bike': 413, 'blocks': 412, 'book': 412, 'doll': 412, 'horse': 412, 'train': 412} 

Oggetti inseriti: ['blocks_869', 'book_395', 'doll_392', 'horse_532', 'train_41', 'ball_821']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 529
Conteggio: 529
{'ball': 414, 'bike': 413, 'blocks': 413, 'book': 413, 'doll': 413, 'horse': 413, 'train': 413} 

Oggetti inseriti: ['bike_329', 'blocks_653', 'book_467', 'doll_387', 'horse_76']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 530
Conteggio: 530
{'ball': 414, 'bike': 414, 'blocks': 414, 'book': 414, 'doll': 414, 'horse': 414, 'train': 413} 

Oggetti inseriti: ['train_432',

Sacchi riempiti: 627
Conteggio: 627
{'ball': 482, 'bike': 482, 'blocks': 481, 'book': 481, 'doll': 481, 'horse': 481, 'train': 481} 

Oggetti inseriti: ['blocks_525', 'book_1021', 'doll_993', 'horse_704', 'train_846', 'ball_977']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 628
Conteggio: 628
{'ball': 483, 'bike': 482, 'blocks': 482, 'book': 482, 'doll': 482, 'horse': 482, 'train': 482} 

Oggetti inseriti: ['bike_54', 'blocks_468', 'book_502']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 629
Conteggio: 629
{'ball': 483, 'bike': 483, 'blocks': 483, 'book': 483, 'doll': 482, 'horse': 482, 'train': 482} 

Oggetti inseriti: ['doll_784', 'horse_361', 'train_820', 'ball_1035']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 630
Conteggio: 630
{'ball': 484, 'bike': 483, 'blocks': 483, 'book': 483, 'doll': 483, 'horse': 483, 'train': 483} 

Oggetti inseriti: ['bike_350', 'blocks_523', 'book_113', 'doll_512']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 631
Conteggio: 631
{'ball': 484, 'bi


Oggetti inseriti: ['horse_33', 'train_941', 'ball_281', 'blocks_253', 'book_604', 'doll_939', 'horse_940', 'train_873']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 735
Conteggio: 735
{'ball': 619, 'blocks': 619, 'book': 619, 'doll': 619, 'horse': 619, 'train': 619} 

Oggetti inseriti: ['ball_492', 'blocks_195', 'book_328', 'doll_591', 'horse_54', 'train_433', 'ball_782', 'blocks_8', 'book_1165', 'doll_706']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 736
Conteggio: 736
{'ball': 621, 'blocks': 621, 'book': 621, 'doll': 621, 'horse': 620, 'train': 620} 

Oggetti inseriti: ['horse_574', 'train_519', 'ball_484', 'blocks_216', 'book_413', 'doll_537', 'horse_11', 'train_877', 'ball_485', 'blocks_914', 'book_1199']
Sacchi fermati alla dogana: 0
Sacchi riempiti: 737
Conteggio: 737
{'ball': 623, 'blocks': 623, 'book': 623, 'doll': 622, 'horse': 622, 'train': 622} 

Oggetti inseriti: ['doll_759', 'horse_251', 'train_889', 'ball_121', 'blocks_751', 'book_1082', 'doll_302', 'horse_712', 'tr

## Giocattoli inseriti

In [113]:
print('Oggetti inseriti: %s' %(sum([len(el) for el in big_bag_ID])))
print('Proporzione oggetti inseriti: %f' %((sum([len(el) for el in big_bag_ID])/len(gift))))

Oggetti inseriti: 7030
Proporzione oggetti inseriti: 0.986805
time: 9.42 ms


## Peso totale dei sacchi

In [99]:
print('Il peso totale dei sacchi risulta: %f lbs' %(sum([sum(el) for el in big_bag_weight])))
print('Il peso medio per sacco risulta: %f lbs' %((sum([sum(el) for el in big_bag_weight]))/len(big_bag_weight)))

Il peso totale dei sacchi risulta: 47638.487441 lbs
Il peso medio per sacco risulta: 47.638487 lbs
time: 12.9 ms


## Controllo dei vincoli sui sacchi

In [94]:
print('Numero di sacchi oltre 50 lbs: %s' %(len([sum(el) for el in big_bag_weight if sum(el)>50])))
print('Numero di sacchi con meno di 3 giocattoli: %s' %(len([el for el in big_bag_ID if len(el) < 3])))

Numero di sacchi oltre 50 lbs: 0
Numero di sacchi con meno di 3 giocattoli: 0
time: 13.9 ms


## Aggiunga dei giocattoli mancanti

In [100]:
#Alcuni giocattoli non sono stati inseriti, ma è possibile che in alcuni sacchi ci sia ancora spazio per loro

for el in diz_freq.keys(): #Per ogni tipologia di oggetto
    while len(globals()[el]) > 0:
        campione = globals()[el].sample(1) #Estrazione casuale
        campione_id = campione.index.values[0]
        campione_toy = campione['Toy'].iloc[0]
        campione_weight = campione['Weight'].iloc[0]
        
        cont = 0
        while cont < len(big_bag_ID): #Per ogni giocattolo controllo se esiste almeno un sacco in cui potrebbe entrare
            if sum(big_bag_weight[cont]) + campione_weight < 50:
                big_bag_weight[cont].append(campione_weight)
                big_bag_ID[cont].append(campione_id)
                big_bag_Toy[cont].append(campione_toy)
                globals()[el].drop(campione.index.values[0], axis = 0, inplace = True)
                diz_freq[el] = diz_freq[el] + 1
                break
            else:
                pass
            
            cont = cont + 1
            
            
        if cont == len(big_bag_ID):
            globals()[el].drop(campione.index.values[0], axis = 0, inplace = True)

time: 2.06 s


## Ricontrollo i valori aggiornati

In [117]:
print('Oggetti inseriti: %s' %(sum([len(el) for el in big_bag_ID])))
print('Proporzione oggetti inseriti: %f' %((sum([len(el) for el in big_bag_ID])/len(gift))))

print('\n')

print('Il peso totale dei sacchi risulta: %f lbs' %(sum([sum(el) for el in big_bag_weight])))
print('Il peso medio per sacco risulta: %f lbs' %((sum([sum(el) for el in big_bag_weight]))/len(big_bag_weight)))

print('\n')

print('Numero di sacchi oltre 50 lbs: %s' %(len([sum(el) for el in big_bag_weight if sum(el)>50])))
print('Numero di sacchi con meno di 3 giocattoli: %s' %(len([el for el in big_bag_ID if len(el) < 3])))

Oggetti inseriti: 7030
Proporzione oggetti inseriti: 0.986805


Il peso totale dei sacchi risulta: 49410.810965 lbs
Il peso medio per sacco risulta: 49.410811 lbs


Numero di sacchi oltre 50 lbs: 0
Numero di sacchi con meno di 3 giocattoli: 0
time: 46.6 ms


## Generalizzazione

L'algoritmo implementato ottiene risultati diversi a seconda del campione estratto ad ogni passaggio. Si possono ottenere dei risultati più generalizzati ripetendo l'operazione di ottimizzazione <i>n</i> volte.
<br>
Utilizzando un <b>random seed</b> per poter replicare i risultati si replica l'algoritmo di ottimizzazzione <i>100</i> volte.

In [124]:
repliche = 2

really_big_bag_ID = []
really_big_bag_Toy = []
really_big_bag_weight = []
bag_count_ID = []
bag_sum_weight = []
bag_time = []

for replica in range(repliche):
    start_time = time.time()
    #Inizializzazione dei dataset
    ball = gift[gift['Toy'] == 'ball'].reset_index(drop = True).set_index('GiftId')
    bike = gift[gift['Toy'] == 'bike'].reset_index(drop = True).set_index('GiftId')
    blocks = gift[gift['Toy'] == 'blocks'].reset_index(drop = True).set_index('GiftId')
    book = gift[gift['Toy'] == 'book'].reset_index(drop = True).set_index('GiftId')
    coal = gift[gift['Toy'] == 'coal'].reset_index(drop = True).set_index('GiftId')
    doll = gift[gift['Toy'] == 'doll'].reset_index(drop = True).set_index('GiftId')
    gloves = gift[gift['Toy'] == 'gloves'].reset_index(drop = True).set_index('GiftId')
    horse = gift[gift['Toy'] == 'horse'].reset_index(drop = True).set_index('GiftId')
    train = gift[gift['Toy'] == 'train'].reset_index(drop = True).set_index('GiftId')

    big_bag_ID = []
    big_bag_Toy = []
    big_bag_weight = []
    cont_bag = 0
    oggetti_inseriti = 0
    data_vuoti = 0
    sacchi_fermati_dogana = 0

    diz_freq = {'ball':0, 'bike':0, 'blocks':0, 'book':0, 'coal':0, 'doll':0, 'gloves':0, 'horse':0, 'train':0}
    while cont_bag < 1000: 
        copy = {}
        copy.update(diz_freq) 
        peso_sacco = 0
        bag_ID = []
        bag_Toy = []
        bag_Weight = []

        while peso_sacco < 50:
            min_toy = min(copy, key=copy.get) 
            if len(bag_ID) > 2 and (copy[min_toy] > (oggetti_inseriti)/(9 - data_vuoti)):
                break
   
            if min(globals()[min_toy]['Weight']) + peso_sacco > 50:
                success = 0
                while success == 0 and len(copy) > 1: 
                    del copy[min_toy]
                    min_toy = min(copy, key=copy.get)
                    if min(globals()[min_toy]['Weight']) + peso_sacco > 50:
                        success == 0
                    else:
                        success == 1

            if len(copy) < 2:
                break

            campione = globals()[min_toy].sample(1) 
            peso_temp = campione['Weight'].iloc[0] 
            if peso_sacco + peso_temp > 50:
                while peso_sacco + peso_temp > 50:
                    campione = globals()[min_toy].sample(1) 
                    peso_temp = campione['Weight'].iloc[0] 

            if campione.index.values[0] not in bag_ID:
                bag_ID.append(campione.index.values[0])
                bag_Toy.append(campione['Toy'].iloc[0])
                bag_Weight.append(campione['Weight'].iloc[0])
                
                diz_freq[min_toy] = diz_freq[min_toy] + 1
                copy[min_toy] = copy[min_toy] + 1

                peso_sacco = peso_sacco + peso_temp
                oggetti_inseriti = oggetti_inseriti + 1

                globals()[min_toy].drop(campione.index.values[0], axis = 0, inplace = True)
                if len(globals()[min_toy]) == 0:
                    data_vuoti = data_vuoti + 1
                    del diz_freq[min_toy]
                    del copy[min_toy]
   
        if len(copy) == 0 and len(bag_ID) < 3:
            #print('Attenzione: il sacco è stato bloccato dalla polizia del Polo Nord! Alcuni bambini non riceveranno il loro regalo :(')
            sacchi_fermati_dogana = sacchi_fermati_dogana + 1

        if len(bag_ID) > 2:
            big_bag_ID.append(bag_ID)
            big_bag_Toy.append(bag_Toy)
            big_bag_weight.append(bag_Weight)
            cont_bag = cont_bag + 1

            #print('Oggetti inseriti: %s' %(bag_ID))
            #print('Sacchi fermati alla dogana: %s' %(sacchi_fermati_dogana))
            #print('Sacchi riempiti: %s' %(len(big_bag_ID)))        
            #print('Conteggio: %s' %(cont_bag))
            #print('%s \n' %(diz_freq))
    
    
    really_big_bag_ID.append(big_bag_ID)
    really_big_bag_Toy.append(big_bag_Toy)
    really_big_bag_weight.append(big_bag_weight)
    bag_count_ID.append(sum([len(el) for el in big_bag_ID])) #Oggetti inseriti ad ogni iterazione
    bag_sum_weight.append((sum([len(el) for el in big_bag_ID])/len(gift))) #Peso totale ad ogni iterazione
    for el in diz_freq.keys(): 
        while len(globals()[el]) > 0:
            campione = globals()[el].sample(1) 
            campione_id = campione.index.values[0]
            campione_toy = campione['Toy'].iloc[0]
            campione_weight = campione['Weight'].iloc[0]

            cont = 0
            while cont < len(big_bag_ID):
                if sum(big_bag_weight[cont]) + campione_weight < 50:
                    big_bag_weight[cont].append(campione_weight)
                    big_bag_ID[cont].append(campione_id)
                    big_bag_Toy[cont].append(campione_toy)
                    globals()[el].drop(campione.index.values[0], axis = 0, inplace = True)
                    diz_freq[el] = diz_freq[el] + 1
                    break
                else:
                    pass

                cont = cont + 1

            if cont == len(big_bag_ID):
                globals()[el].drop(campione.index.values[0], axis = 0, inplace = True)
            
            
    #Aggiornamento delle liste dopo l'inclusione dei giocattoli non inseriti in un primo momento
    really_big_bag_ID.append(big_bag_ID)
    really_big_bag_Toy.append(big_bag_Toy)
    really_big_bag_weight.append(big_bag_weight)
    bag_count_ID.append(sum([len(el) for el in big_bag_ID])) 
    bag_sum_weight.append((sum([len(el) for el in big_bag_ID])/len(gift))) 
    
    end_time = time.time()
    bag_time.append(end_time - start_time)  
    
    print(replica)

0
1
time: 1min 50s


In [130]:
bag_sum_weight

[0.9195676586187536,
 0.9869455362156092,
 0.9192869174620999,
 0.9863840539023021]

time: 36.2 ms
